In [1]:
import pandas as pd
import numpy as np
import os
import h5py

/dvmm-filer2/users/jiawei/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
combine_file = h5py.File('data_mask_maximum.h5','r')
print combine_file.keys()
data = combine_file['data'][:]
mask = combine_file['mask'][:]
position = combine_file['position'][:]
maximum = combine_file['maximum'].value
combine_file.close()
print data.shape,mask.shape,position.shape,mask.mean()
print maximum,mask.mean(),data.max(),data.mean(),data.min()

[u'data', u'mask', u'maximum', u'position']
(186, 6624) (186, 6624) (186, 2) 0.9157041
25.585367 0.9157041 25.585367 2.992725 0.0


In [3]:
mask_miss = 1.0-mask
mask_time,mask_delta = np.zeros_like(mask),np.zeros_like(mask)
mask_time[:,0] = mask_miss[:,0]
for cam in range(186):
    for time in range(1,6624):
        if mask_miss[cam,time] == 1:
            mask_time[cam,time] = mask_time[cam,time-1]+1
    mask_delta[cam,:] = mask_time[cam,:]
    while(time>0):
        if mask_time[cam,time] > 1:
            length = int(mask_time[cam,time]-1)
            mask_time[cam,time-length:time] = mask_time[cam,time]
            mask_delta[cam,time-length:time] = 0
            time -= length
        time -= 1
delta_seq = mask_delta[mask_delta != 0]
delta_seq = np.sort(delta_seq)
delta_burst = delta_seq[delta_seq<=134]
print delta_burst.shape
delta_burst = delta_burst[delta_burst > 1]
print delta_burst.shape
print delta_burst.mean(),delta_burst.std(),mask_miss.mean()

(24290,)
(5887,)
6.350773 9.809643 0.08429594


In [71]:
burst_mean,burst_std,mask_burst,ratio = delta_burst.mean(),delta_burst.std(),np.zeros_like(mask_miss),0.618
for cam in range(186):
    time = 0
    while(time<6624):
        if np.random.uniform(size=1) < ratio:
            length = int(np.random.normal(burst_mean, burst_std, 1))
            if length > 0:
                mask_burst[cam,time:time+length] = 1.0
                time += length
            else:
                time += 1
        else:
            time += 1
move = mask_burst+mask_miss
move[move>1] = 1
move = 1.0-move
for i in range(6624-48*12+1):
    if move[:,i:i+48*12].mean() == 0:
        print i
print 'missing rate:', 1.0-move.mean()

combine_file = h5py.File('burst_90.h5','w')
combine_file['mask'] = move
combine_file.close()

missing rate: 0.9009605050086975


In [79]:
combine_file = h5py.File('burst_20.h5','r')
move = combine_file['mask'][:]
combine_file.close()
print move.mean(),move.max(),move.min()
for i in range(6624-48*12+1):
    if move[:,i:i+48*12].mean() == 0:
        print i
temp = move[move!=0]

print temp[temp!=1]

0.7999438 1.0 0.0
[]
